In [ ]:
!pip install pymorphy2
!pip install razdel

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pymorphy2
from razdel import sentenize
from razdel import tokenize
import torch
from transformers import AutoTokenizer, AutoModel
import re
import gc
import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny", output_hidden_states = True)
model.eval()

In [ ]:
df = pd.read_csv("/kaggle/input/newsdataset1csv/1.csv")
df.dropna(inplace=True)

In [ ]:
pm = pymorphy2.MorphAnalyzer()

In [ ]:
with open('/kaggle/input/ws353-full/ws353_full.pkl', 'rb') as f:
    ws_full = pickle.load(f)

In [ ]:
contexts = {}

In [ ]:
from numpy.random import default_rng
rng = default_rng()

In [ ]:
model.to('cuda')

max_len = 90
n_contexts = 50
print(f'Number of contexts = {n_contexts}')
i = 1

for key, value in tqdm.tqdm(ws_full.items()):
    print(f'Iteration {i} / {len(ws_full)}')
    i += 1    
    vector = [] # здесь будут храниться эмбеддинги, то есть его размер будет (n_contexts, 312)
    n = 0
    size = n_contexts if n_contexts <= len(value) else len(value)
    indices = rng.choice(len(value), size, replace=False)
    batch = []
    words = []
    for index in indices:
        idx, n_sent, w_idx = value[index]
        if n >= n_contexts:
            break
        n += 1
        string_to_parse = df.iloc[idx].values[1]
        string_to_parse = re.sub(' *\n\n *', '. ', string_to_parse)
        new_str = re.sub(' +|\n *\n| *\xa0| *\n', ' ', string_to_parse)
        sent = list(sentenize(new_str))[n_sent]
        batch.append(sent.text)
        words.append(list(tokenize(sent.text.lower()))[w_idx])
    with torch.no_grad():
        d = tokenizer(batch, return_offsets_mapping=True, return_tensors='pt', return_attention_mask=False,
                          return_token_type_ids=False, padding=True, truncation=True, max_length=80)
        offset_mapping = d.pop('offset_mapping')
        input_ids = d['input_ids'].to('cuda')
        sent_vec = model(input_ids).hidden_states
        layers = torch.tensor(np.array([i.to('cpu').numpy() for i in sent_vec[1:]]), dtype=torch.float32)
#         layers = torch.stack(sent_vec)[1:]
        for k in range(layers.shape[1]):
            start = np.where(offset_mapping[k, :, 0] == words[k].start)[0]
            if (start.shape[0] == 0):
                continue
            start = start[0]
            stop = np.where(offset_mapping[k, :, 1] == words[k].stop)[0]
            if (stop.shape[0] == 0):
                continue
            stop = stop[0]
            vector.append(layers[:, k, start:stop+1, :].mean(axis=1))
    vector = torch.stack(vector)
    idxs = rng.choice(vector.shape[0], min(n_contexts, vector.shape[0]), replace=False)
    contexts[key] = vector[idxs].mean(axis=0)